In [ ]:
from keras.models import Sequential
import keras.layers
import keras.utils
from keras.layers import Conv2D, Activation
from keras.layers import MaxPool2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.layers.core import Activation
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
folder_path_train ="/content/drive/MyDrive/HMD_project/train.jsonl"
folder_path_dev ="/content/drive/MyDrive/HMD_project/dev.jsonl"
train = pd.read_json(folder_path_train,lines=True)


val=pd.read_json(folder_path_dev,lines=True)

print(train.head(1))
print(train.label.value_counts())

# Read train and validation dataset information from json file

In [ ]:
# for local machine
# train=pd.read_json('/media/adb/Data/Sabudh_training_DS10/Hateful_meme_Sangita/HMD_code/train.jsonl',lines=True)
# val=pd.read_json('/media/adb/Data/Sabudh_training_DS10/Hateful_meme_Sangita/HMD_code/dev.jsonl',lines=True)


# Load image from training dataset
# N.B: All images are resized (224x224x3) using PIL and saved in drive due to limitations in space

In [ ]:

img_path=train.img
img_label=train.label
l=len(img_path)

x=[]
y=[]

for i in range(l):
    print(i)
  
    img_=Image.open("/content/drive/MyDrive/HMD_project/"+str(img_path[i])+".png")
    img_files=np.array(img_)
    # img_files=(img_files/255.0).astype(np.float16)
  
    
    if img_files.shape==(224,224,4):
        img_files=img_files[:,:,0:3]
        
    x.append(img_files)
    y.append(img_label[i])
x_arr=np.array(x)
x_arr.shape


# Load images from validation dataset

In [ ]:
img_path=val.img
img_label=val.label
l=len(img_path)

x_val=[]
y_val=[]

for i in range(l):
  
    img_=Image.open("/content/drive/MyDrive/HMD_project/"+str(img_path[i])+".png")
    img_files=np.array(img_)
    # img_files=(img_files/255.0).astype(np.float16)
    

    if img_files.shape==(224,224,4):
        img_files=img_files[:,:,0:3]
        
    x_val.append(img_files)
    y_val.append(img_label[i])
x_val=np.array(x_val)
y_val=np.array(y_val)
x_val.shape

# for data balancing in training dataset create augmented images from hate images and save the images in a folder

In [ ]:

lst=train['label']==1  # hate images
list_1=train[lst]

print("Total no. of hate images in training dataset",list_1.label.value_counts())
id_1=np.random.randint(0,len(list_1),size=1200) # create id for 1200 random images

len(id_1)

In [ ]:

l=len(id_1)
id_lst=[]
aug_text=[]

for i in range(l):
    img=Image.open('/content/drive/MyDrive/HMD_project/'+str(list_1.iloc[id_1[i]].img)+".png")
    id=list_1.iloc[id_1[i]].id

    id_img="augmented/Aug_"+str(id)+".png"
    id_lst.append(id_img)   # save path of augmented images to a list for further loading

    new_text=list_1.iloc[id_1[i]].text # make a list of text for corresponding images
    aug_text.append(new_text)

   
    con=np.random.randint(0,2)

    if con==0:
        data_aug= keras.layers.RandomFlip(mode="horizontal")
        a=data_aug(img)
    else:
        data_aug= keras.layers.RandomRotation(factor=(0.02,0.1))
        a=data_aug(img)

    pil_img = keras.utils.array_to_img(a)
    
    # save the augmented images in a folder named Augmented_label1
    pil_img.save('/content/drive/MyDrive/HMD_project/augmented/Aug_'+str(id)+'.png')


In [ ]:
# save the list of text in a txt file for text processing
file=open('/content/drive/MyDrive/HMD_project/text_aug_norm.txt','w')
for i in aug_text:
    file.write(i+'\n')


# read the augmeted images from the folder using list of paths

In [ ]:
x_train_aug=[]
l=len(id_lst)

for i in range(l):
    img_aug=Image.open('/content/drive/MyDrive/HMD_project/'+str(id_lst[i]))
    
    imp_file=np.array(img_aug)
    # imp_file=(imp_file/255.0).astype(np.float16)
  
    x_train_aug.append(imp_file)

y_train_aug=np.ones(len(x_train_aug))

print('Data samples in Augmented dataset:',len(x_train_aug))

In [ ]:
print((x[0].dtype))
print(x_train_aug[0].dtype)

In [ ]:
x.extend(x_train_aug)
y.extend(y_train_aug)

In [ ]:
x_train=np.array(x)
y_train=np.array(y)

print(x_train.shape)
print(y_train.shape)

# VGG16 without top layer

In [ ]:
model_input=x_train
model_input_val=x_val

In [ ]:
cnn=Sequential()
#1
cnn.add(Conv2D(input_shape=(224,224,3),data_format='channels_last',filters=64,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(Conv2D(filters=64,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#2
cnn.add(Conv2D(filters=128,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(Conv2D(filters=128,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#3
cnn.add(Conv2D(filters=256,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(Conv2D(filters=256,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(Conv2D(filters=256,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#4
cnn.add(Conv2D(filters=512,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(Conv2D(filters=512,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(Conv2D(filters=512,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#5
cnn.add(Conv2D(filters=512,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(Conv2D(filters=512,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(Conv2D(filters=512,padding='same',kernel_size=(3,3),activation='relu'))
cnn.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


# weights from VGG16 model
vggmodel=(VGG16(include_top=False, weights='imagenet'))
vggmodel.save('/content/drive/MyDrive/HMD_project/trained_model.h5')
cnn.load_weights('/content/drive/MyDrive/HMD_project/trained_model.h5')

for layer in cnn.layers:
    layer.trainable=False

cnn.add(Flatten())

# save cnn outputs for both train and test dataset

In [ ]:

output=cnn.predict(model_input,batch_size=50)
output.shape
# f=open('/content/drive/MyDrive/HMD_project/cnn_train_op.npy', 'wb')
np.save('/content/drive/MyDrive/HMD_project/cnn_train_op_norm.npy',output)




In [ ]:

output_val=cnn.predict(model_input_val,batch_size=50)
print(output_val.shape)
# f=open('/content/drive/MyDrive/HMD_project/cnn_val_op.npy', 'wb')
np.save('/content/drive/MyDrive/HMD_project/cnn_val_op_norm.npy',output_val)


# Autoencoder

# Data normalization 

In [ ]:
model_input=output   # CNN output
model_input_norm=model_input/np.max(model_input)
model_input=model_input_norm.astype(np.float16)

# design the encoder and decoder segments by selecting the hyper-parameters

In [ ]:
model=Sequential()
from keras.layers import BatchNormalization

from keras.layers import Dropout

# model.add(keras.Input(shape=(output.shape[-1],)))

model.add(Dense(500,input_shape=(model_input.shape[-1],),activation='relu',name="encoder1"))
model.add(Dropout(0.25))
# model.add(BatchNormalization())
model.add(Dense(200,activation='relu',name="encoder2"))
model.add(Dropout(0.1))

model.add(Dense(200,activation='relu',name="encoder3"))
# model.add(BatchNormalization())
# model.add(Dropout(0.1))
model.add(Dense(200,activation='relu',name="decoder1"))
model.add(Dropout(0.1))
# model.add(BatchNormalization())
model.add(Dense(500,activation='relu',name="decoder2"))
# model.add(BatchNormalization())

model.add(Dropout(0.25))
model.add(Dense(model_input.shape[-1],activation='relu',name="decoder3"))


model.summary()

In [ ]:

model.compile(loss='mse',optimizer='adam')
history=model.fit(model_input,model_input,epochs=200,batch_size=128,shuffle=True,validation_split=0.2)
print(history.history.keys())

In [ ]:
plt.figure()
plt.plot(history.history['loss'],label='train_loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.legend()
plt.show()


# save image embedding (taken from encoder output) and label in a numpy file

In [ ]:
import tensorflow as tf
from keras import backend as K

layer_ID = 4
get_output = K.function([model.get_layer(index=0).input], model.get_layer(index=layer_ID).output)

embedded_train_img = get_output([model_input])  # input_data is a numpy array
print((embedded_train_img).shape)

train_img=np.append(embedded_train_img,y_train,axis=1)

np.save('/content/drive/MyDrive/HMD_project/HMD_classifier/embedding_train_img_norm.npy',train_img)
print(train_img.shape)



In [ ]:
model_input_val_norm=output_val/np.max(output_val)

model_input_val=model_input_val_norm.astype(np.float16)
model_input_val.shape

In [ ]:
y_val=y_val.reshape(y_val.size,1)

embedded_val_img = get_output([model_input_val])  # input_data is a numpy array
print((embedded_val_img).shape)

val_img=np.append(embedded_val_img,y_val,axis=1)
np.save('/content/drive/MyDrive/HMD_project/HMD_classifier/embedding_val_img_norm.npy', val_img)
print(val_img.shape)
